In [3]:
%pip install cpmpy

70.86s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
from cpmpy import *

e = 0 # value for empty cells
given = np.array([
    [e, e, e,  2, e, 5,  e, e, e],
    [e, 9, e,  e, e, e,  7, 3, e],
    [e, e, e,  e, e, 9,  e, 6, e],

    [2, e, e,  e, e, e,  4, e, 9],
    [e, e, e,  e, 7, e,  e, e, e],
    [6, e, 9,  e, e, e,  e, e, 1],

    [e, 8, e,  4, e, e,  1, e, e],
    [e, 6, 3,  e, e, e,  e, 8, e],
    [e, e, e,  6, e, 8,  e, e, e]])

puzzle = intvar(1, 9, shape=given.shape, name="puzzle")

# we create a model with the row/column constraints
model = Model(
    # Constraints on rows and columns
    [AllDifferent(row) for row in puzzle],
    [AllDifferent(col) for col in puzzle.T], # numpy's Transpose
)

# we extend it with the block constraints
# Constraints on blocks
for i in range(0,9, 3):
    for j in range(0,9, 3):
        model += AllDifferent(puzzle[i:i+3, j:j+3]) # python's indexing

# Constraints on values (cells that are not empty)
model += (puzzle[given!=e] == given[given!=e]) # numpy's indexing

if model.solve():
    print(puzzle.value())
else:
    print("No solution found")



[[7 4 6 2 3 5 9 1 8]
 [1 9 5 8 6 4 7 3 2]
 [3 2 8 7 1 9 5 6 4]
 [2 3 1 5 8 6 4 7 9]
 [8 5 4 9 7 1 6 2 3]
 [6 7 9 3 4 2 8 5 1]
 [5 8 7 4 2 3 1 9 6]
 [4 6 3 1 9 7 2 8 5]
 [9 1 2 6 5 8 3 4 7]]


# Ideas

- We have `D` devices, belonging to `B` buildings, and `E` engineers
- Each building and engineer has a location. 
- We compute a global distance matrix where the first `E` rows and columns are related to the `E` engineers 
- Therefore, the shape of the distance matrix is `E+D,E+D`

In [35]:
num_devices = 4
num_engineers = 2
distance_matrix = np.array([
        [0, 100, 100, 100, 100, 300],
        [100, 0, 100, 100, 100, 100],
        [100, 100, 0, 100, 100, 100],
        [100, 100, 100, 0, 100, 100],
        [100, 100, 100, 100, 0, 100],
        [100, 100, 100, 100, 100, 0]
])

ideal_target = int(num_devices/num_engineers)

# Set to True if the engineer is assigned to the device
assignments = boolvar(name="assignments", shape=(num_engineers, num_devices))

model = Model()

# Add a constraint so that each device is assigned to exactly one engineer
model += [sum(assignments[:, i]) == 1 for i in range(num_devices)]

# Minimize the difference between the ideal target and the actual number of devices assigned to each engineer 
model.minimize(sum(
        [abs(ideal_target - sum(assignments[i, :])) for i in range(num_engineers)]
))

# Minimize the distance between the engineer and the devices
model.minimize(
        sum( # For each engineer
        distance_matrix[0, num_engineers:][assignments[0] == True]
                #[sum(distance_matrix[i, num_engineers:][assignments[i, :] > 1]) for i in range(num_engineers)]
        )
)


print(model.solve(), model.status())
print(assignments.value())

IndexError: arrays used as indices must be of integer (or boolean) type

In [14]:
engIndex = 0
devices = np.array([0, 0, 1, 1])
distance_matrix[engIndex,num_engineers:]

# Keep only the distances where the device is 1
distances = distance_matrix[engIndex,num_engineers:]
print(distances[devices == 1])

[100 300]


In [ ]:
# results = assignments.value()
# sum(abs(ideal_target - results.sum(axis=1)))
test=[[0, 0, 1, 1, 1, 0], [1, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0]]
np.sum(np.abs(ideal_target -  np.sum(test, axis=1)))

2